In [19]:
import numpy as np
import pandas as pd
import yfinance as yf
import ta

import warnings
warnings.filterwarnings("ignore")

In [20]:
def interval_split(dt):
    #for date_time column of "sXXdXXpXX"
    f2, f3 = dt.find("d"), dt.find("p")
    return [int(dt[1:f2]), int(dt[f2+1:f3]), int(dt[f3+1:])]

def date_split(dt):
    #for date_time column of "sXXdXX"
    f2 = dt.find("d")
    return [int(dt[1:f2]), int(dt[f2+1:])]


def add_interval(df):
    df_interval_data = np.vstack(df.date_time.apply(lambda x: interval_split(x)))
    df[["asset", "day", "interval"]] = df_interval_data
    return df

def add_date(df):
    df_date_data = np.vstack(df.date_time.apply(lambda x: date_split(x)))
    df[["asset", "day"]] = df_date_data
    return df

#Features in Consideration
def merge_columns(market, fundamental, returns=pd.DataFrame([0])):
    market = add_interval(market)
    fundamental = add_date(fundamental)
    df = pd.merge(market, fundamental, left_on=["asset", "day"], right_on=["asset", "day"])
    if returns.any()[0]:    #check if returns is a non-empty dataframe
        returns = add_date(returns)
        df = pd.merge(df, returns, left_on=["asset", "day"], right_on=["asset", "day"])
    # Note: This Step Will Mean that Metrics regarding
    return df

print("Ran")

Ran


# Paths and Load CSV (Modify Path Here)

In [8]:
path = '''../round1/'''
fn_train_fund = path + '''first_round_train_fundamental_data.csv'''
fn_train_market = path + '''first_round_train_market_data.csv'''
fn_train_returns = path + '''first_round_train_return_data.csv'''

df_train_fund = pd.read_csv(fn_train_fund)
df_train_market = pd.read_csv(fn_train_market)
df_train_returns = pd.read_csv(fn_train_returns)

df = merge_columns(df_train_market, df_train_fund, df_train_returns)
df

,date_time_x,open,close,high,low,volume,money,asset,day,interval,date_time_y,turnoverRatio,transactionAmount,pe_ttm,pe,pb,ps,pcf,date_time,return
0,s0d1p1,25.0164,24.9557,25.1378,24.9436,642243.0,1.610060e+07,0,1,1,s0d1,3.6794,17229.0,34.4425,32.3029,4.9425,3.8180,-578.7700,s0d1,-0.026877
1,s0d1p2,24.9557,24.8465,24.9557,24.8101,797872.0,1.984058e+07,0,1,2,s0d1,3.6794,17229.0,34.4425,32.3029,4.9425,3.8180,-578.7700,s0d1,-0.026877
2,s0d1p3,24.8465,24.7008,24.8586,24.6644,1234546.0,3.056567e+07,0,1,3,s0d1,3.6794,17229.0,34.4425,32.3029,4.9425,3.8180,-578.7700,s0d1,-0.026877
3,s0d1p4,24.7008,24.7615,24.7858,24.7008,533023.0,1.318848e+07,0,1,4,s0d1,3.6794,17229.0,34.4425,32.3029,4.9425,3.8180,-578.7700,s0d1,-0.026877
4,s0d1p5,24.7615,24.7251,24.7615,24.7008,565000.0,1.396949e+07,0,1,5,s0d1,3.6794,17229.0,34.4425,32.3029,4.9425,3.8180,-578.7700,s0d1,-0.026877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2694595,s53d998p46,9.9046,9.8561,9.9046,9.8561,194291.0,1.919144e+06,53,998,46,s53d998,1.2933,9681.0,10.6513,10.6513,1.9956,0.4073,134.5467,s53d998,-0.003662
2694596,s53d998p47,9.8561,9.9046,9.9046,9.8561,128516.0,1.270606e+06,53,998,47,s53d998,1.2933,9681.0,10.6513,10.6513,1.9956,0.4073,134.5467,s53d998,-0.003662
2694597,s53d998p48,9.8925,9.8925,9.9046,9.8682,194832.0,1.926786e+06,53,998,48,s53d998,1.2933,9681.0,10.6513,10.6513,1.9956,0.4073,134.5467,s53d998,-0.003662
2694598,s53d998p49,9.8925,9.9410,9.9410,9.8803,344046.0,3.407865e+06,53,998,49,s53d998,1.2933,9681.0,10.6513,10.6513,1.9956,0.4073,134.5467,s53d998,-0.003662


In [15]:
#just a base to be implemented with for loop
asset_type = 19

df_a = df[df['asset'] == asset_type]
df_test = df_a.reset_index()
df_test.rename({'index':'old_index'}, axis=1, inplace=TrueW)
df_test
df_test['%K'] = ta.momentum.stoch(df_test.high, df_test.low, df_test.close, window=14, smooth_window=3)
df_test['%D'] = df_test['%K'].rolling(3).mean()
df_test['RSI'] = ta.momentum.rsi(df_test.close, window=14)
df_test['MACD'] = ta.trend.macd_diff(df_test.close)
df_test


,old_index,date_time_x,open,close,high,low,volume,money,asset,day,...,pe,pb,ps,pcf,date_time,return,%K,%D,RSI,MACD
0,950,s19d1p1,7.3799,7.4406,7.4406,7.3799,947409.0,7.022792e+06,19,1,...,32.9395,3.4601,5.1796,-40.8809,s19d1,0.018074,NaN,NaN,NaN,NaN
1,951,s19d1p2,7.4285,7.3799,7.4285,7.3435,1977718.0,1.460695e+07,19,1,...,32.9395,3.4601,5.1796,-40.8809,s19d1,0.018074,NaN,NaN,NaN,NaN
2,952,s19d1p3,7.3799,7.3799,7.4042,7.3799,1230380.0,9.089325e+06,19,1,...,32.9395,3.4601,5.1796,-40.8809,s19d1,0.018074,NaN,NaN,NaN,NaN
3,953,s19d1p4,7.3678,7.3314,7.3799,7.3314,1185851.0,8.722181e+06,19,1,...,32.9395,3.4601,5.1796,-40.8809,s19d1,0.018074,NaN,NaN,NaN,NaN
4,954,s19d1p5,7.3314,7.3556,7.3556,7.3192,620319.0,4.552995e+06,19,1,...,32.9395,3.4601,5.1796,-40.8809,s19d1,0.018074,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49895,2692895,s19d998p46,7.9383,7.9625,7.9747,7.9383,188615.0,1.499179e+06,19,998,...,41.7459,2.8802,2.4964,71.3265,s19d998,-0.059635,18.127341,16.036955,45.211764,-0.002935
49896,2692896,s19d998p47,7.9747,7.9504,7.9868,7.9504,228689.0,1.821195e+06,19,998,...,41.7459,2.8802,2.4964,71.3265,s19d998,-0.059635,9.063670,9.063670,43.246399,-0.004396
49897,2692897,s19d998p48,7.9504,7.9504,7.9747,7.9383,300498.0,2.391914e+06,19,998,...,41.7459,2.8802,2.4964,71.3265,s19d998,-0.059635,12.461380,13.217464,43.246399,-0.005076
49898,2692898,s19d998p49,7.9625,7.9504,7.9625,7.9383,508600.0,4.043707e+06,19,998,...,41.7459,2.8802,2.4964,71.3265,s19d998,-0.059635,12.461380,11.328810,43.246399,-0.005219


In [23]:
def gettrigger(df, lags, buy=True):
    dfx = pd.DataFrame()
    for i in range(1, 4):
        if buy:
            mask = (df_test['%K'].shift(i) < 20) & (df_test['%D'].shift(i) < 20)
        else:
            mask = (df_test['%K'].shift(i) > 80) & (df_test['%D'].shift(i) < 80)
        dfx = dfx.append(mask, ignore_index=True)
    return dfx.sum(axis=0)

def get_total_return(df, asset_type):
    df_a = df[df['asset'] == asset_type]
    df_test = df_a.reset_index()
    df_test.rename({'index':'old_index'}, axis=1, inplace=True)
    df_test
    df_test['%K'] = ta.momentum.stoch(df_test.high, df_test.low, df_test.close, window=14, smooth_window=3)
    df_test['%D'] = df_test['%K'].rolling(3).mean()
    df_test['RSI'] = ta.momentum.rsi(df_test.close, window=14)
    df_test['MACD'] = ta.trend.macd_diff(df_test.close)
    df_test['BuyTrigger'] = np.where(gettrigger(df, 4, True), 1, 0)
    #df_test['SellTrigger'] = np.where(gettrigger(df, 4, False), 1, 0)
    df_test['Buy'] = np.where((df_test['BuyTrigger']) & 
                            (df_test['%K'].between(20,80)) &
                            (df_test['%D'].between(20,80)) &
                            (df_test['RSI'] < 50) &
                            (df_test['MACD'] < 0), 1, 0)
    return df_test[(df_test['interval'] == 50) & (df_test['Buy'] == 1)]['return'].sum()

In [25]:
sum = 0
for i in range(54):
    sum += get_total_return(df, i)

print(sum)

7.8772504525556055
